In [2]:
import pandas as pd
import numpy as np

## Data Cleaning and Transformation

Climate opinion data

In [24]:
yale_climate = pd.read_csv('data/yale_climate.csv')
yale_climate.head()

,geotype,geoid,geoname,varname,x2010,x2011,x2012,x2013,x2014,x2015,x2016,x2017,x2018,x2019,x2020,x2021,x2022,x2023
0,state,1,Alabama,affectweather,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.66,49.22,53.33,53.64,49.63,50.28
1,state,2,Alaska,affectweather,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.48,54.94,60.92,62.25,59.17,60.66
2,state,4,Arizona,affectweather,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.32,58.40,63.55,64.22,61.73,63.12
3,state,5,Arkansas,affectweather,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.17,50.78,55.72,55.89,51.26,52.43
4,state,6,California,affectweather,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.21,64.51,69.28,70.14,67.47,68.39


In [25]:
yale_climate = yale_climate[yale_climate['geotype'] == 'state']
yale_climate = yale_climate[['geoname','varname', 'x2018', 'x2019', 'x2020', 'x2021', 'x2022', 'x2023']]
yale_climate['geoname'] = yale_climate['geoname'].str.upper() #consistent state names across both datasets
yale_climate.head()

,geoname,varname,x2018,x2019,x2020,x2021,x2022,x2023
0,ALABAMA,affectweather,52.66,49.22,53.33,53.64,49.63,50.28
1,ALASKA,affectweather,57.48,54.94,60.92,62.25,59.17,60.66
2,ARIZONA,affectweather,60.32,58.40,63.55,64.22,61.73,63.12
3,ARKANSAS,affectweather,53.17,50.78,55.72,55.89,51.26,52.43
4,CALIFORNIA,affectweather,67.21,64.51,69.28,70.14,67.47,68.39


In [26]:
melted = pd.melt(yale_climate, id_vars=['geoname', 'varname'], var_name='year', value_name='percentage')
melted['year'] = melted['year'].str.replace('x','').astype(int)
yale_climate = melted
yale_climate.head()

,geoname,varname,year,percentage
0,ALABAMA,affectweather,2018,52.66
1,ALASKA,affectweather,2018,57.48
2,ARIZONA,affectweather,2018,60.32
3,ARKANSAS,affectweather,2018,53.17
4,CALIFORNIA,affectweather,2018,67.21


Storm data

In [27]:
df = pd.read_csv('data/storm_dataset2023.csv')
df.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,202310,25,230,202310,27,551,186682,1145781,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",Public reports 7.5 inches at Black Tiger Bay C...,CSV
1,202310,25,230,202310,27,1437,186682,1145783,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",Local Police Department relays storm total sno...,CSV
2,202310,25,230,202310,27,1126,186682,1145784,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",Public reports 10 inches of storm total snowfa...,CSV
3,202310,25,230,202310,27,1301,186682,1145796,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",Emergency Manager reports 6 inches of storm to...,CSV
4,202310,25,230,202310,27,600,186682,1145884,NORTH DAKOTA,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"In late October, a winter storm dumped heavy s...",CoCoRaHS Station ND-GF-23 reports 8.8 inches o...,CSV


In [38]:
unique = df.drop_duplicates(subset=['STATE_FIPS', 'EPISODE_ID']) #there are multiple events for the same episode, this removes them
agg_funcs = {
    'EVENT_ID': 'count',
    'YEAR': 'first',
}
by_state = unique.groupby('STATE').agg(agg_funcs).reset_index()
by_state = by_state.rename({'EVENT_ID': 'COUNT'}, axis=1)
events = unique[['STATE', 'EVENT_TYPE', 'YEAR', 'EPISODE_ID', 'EPISODE_NARRATIVE']] #only columns we might use
events

,STATE,EVENT_TYPE,YEAR,EPISODE_ID,EPISODE_NARRATIVE
0,NORTH DAKOTA,Heavy Snow,2023,186682,"In late October, a winter storm dumped heavy s..."
6,MISSISSIPPI,Thunderstorm Wind,2023,180848,"In the afternoon and evening of the 24th, clus..."
7,MINNESOTA,Drought,2023,186724,D2 Drought continued from September into early...
9,ILLINOIS,Thunderstorm Wind,2023,184619,A vigorous short-wave trough interacted with a...
11,VIRGINIA,Thunderstorm Wind,2023,185704,Scattered severe thunderstorms in advance of a...
...,...,...,...,...,...
74851,TEXAS,Thunderstorm Wind,2023,186385,Thunderstorms developed along a cold front as ...
74858,NEW HAMPSHIRE,Flash Flood,2023,184741,An area of low pressure moved northeast from M...
74863,TEXAS,Wildfire,2023,185514,"While weather conditions were pretty tame, we ..."
74864,TEXAS,Wildfire,2023,185515,Hot conditions continued for the eastern Panha...


In [43]:
event_dfs = []
by_state_dfs = []
def load_storm(year):
    df = pd.read_csv(f'data/storm_dataset{year}.csv')
    unique = df.drop_duplicates(subset=['STATE_FIPS', 'EPISODE_ID']) #there are multiple events for the same episode, this removes them
    agg_funcs = {
        'EVENT_ID': 'count',
        'YEAR': 'first',
    }
    by_state = unique.groupby('STATE').agg(agg_funcs).reset_index()
    by_state = by_state.rename({'EVENT_ID': 'COUNT'}, axis=1)
    events = unique[['STATE', 'EVENT_TYPE', 'YEAR', 'EPISODE_ID', 'EPISODE_NARRATIVE']] #only columns we might use
    event_dfs.append(events)
    by_state_dfs.append(by_state)

In [44]:
years = [2018 + i for i in range(6)]
for year in years:
    load_storm(year)

In [51]:
#note: some 'states' on here are not actual states (like Lake Superior) - we will ignore these for now
events_df = pd.concat(event_dfs, axis=0)
event_count_df = pd.concat(by_state_dfs, axis=0)
event_count_df

,STATE,COUNT,YEAR
0,ALABAMA,149,2018
1,ALASKA,68,2018
2,AMERICAN SAMOA,19,2018
3,ARIZONA,277,2018
4,ARKANSAS,191,2018
...,...,...,...
62,VIRGINIA,212,2023
63,WASHINGTON,107,2023
64,WEST VIRGINIA,131,2023
65,WISCONSIN,211,2023


In [53]:
events_df_merged = events_df.merge(yale_climate, how='inner', left_on=['STATE', 'YEAR'], right_on=['geoname','year'])
events_df_merged = events_df_merged.drop(columns=['geoname', 'year']) #data for each event type along with yale climate data

In [55]:
event_count_merged = event_count_df.merge(yale_climate, how='inner', left_on=['STATE', 'YEAR'], right_on=['geoname', 'year'])
event_count_merged = event_count_merged.drop(columns=['geoname', 'year']) #data with number of events for each year along with the yale climate data

In [49]:
#TODO identify questions and filter dfs for each question

## EDA

## Analysis